In [1]:
import numpy as np
import pandas as pd
import nltk
import collections
import re
import string
from nltk.tokenize import sent_tokenize, word_tokenize

### Задание 2

Учитывание порядка предшествующих слов

In [2]:
def ngrams(sentense: list, n: int):
    '''
    sentense: list 
    n: ngram
    
    Выводит сначала 1-gram, потом 2-gram и.т.д - удобно
    
    >>> test = ['cat','sat','on']
    >>> ngrams(test, 2)
    <<< ['cat', 'sat', 'on', 'cat sat', 'sat on', 'cat sat on']
    '''
    
    result = []
    fullgram = len(sentense)
    for i in range(fullgram):
        if i == n:
            break
        for j in range(fullgram-i):
            result.append(' '.join(sentense[j:i+1+j]).rstrip('.'))
    return result

def text_prepare(text):
    """
        text: a string

        return: modified string
    """
    # Перевести символы в нижний регистр
    text = text.lower()

    # Заменить символы пунктуации на пробелы
    text = re.sub(r'[{}]'.format(string.punctuation), '', text)

    # Удалить "плохие" символы
    text = re.sub('[^A-Za-z0-9 ]', '', text)
    return text

#### Test Text

In [3]:
text = '''
Backgammon is one of the oldest known board games. 
Its history can be traced back nearly 5,000 years to archeological discoveries in the Middle East. 
It is a two player game where each player has fifteen checkers 
which move between twenty-four points according to the roll of two dice.
Backgammon do board games every, night fight it in. 
'''

### Our text: Import

In [3]:
file_name = 'new.txt'
with open(file_name, 'r', encoding='utf-8') as file:
    text = file.readlines()
text = ''.join(text)

### Preprocess - ngrams = 4

In [4]:
sentences = nltk.sent_tokenize(text)
words_sentences = [sentence.lower().split() for sentence in sentences]

In [5]:
n = 4
result = [ngrams(snt, n) for snt in words_sentences]
flatten = lambda l: [item for sublist in l for item in sublist]
words = flatten(result)

In [12]:
prepared_words = [text_prepare(word) for word in words]

In [13]:
count = collections.Counter(prepared_words)

### Full-Matrix 

need fix

In [15]:
data = pd.DataFrame(data = count.items(), columns = ['words', 'nu'])
data['len'] = data['words'].apply(lambda x: len(x.split()))

#### 3 words back

In [16]:
k = n
ngram_4 = data[data['len']==k].iloc[:]
for i in range(k):
    ngram_4[f'w_{k-i-1}'] = ngram_4['words'].apply(lambda x: x.split()[i])
ngram_4[f'log_P(w_k-3|w_k)'] = np.log(ngram_4['nu'] / ngram_4['nu'].sum())
ngram_4.head()

,words,nu,len,w_3,w_2,w_1,w_0,log_P(w_k-3|w_k)
87,it goes without saying,1,4,it,goes,without,saying,-6.09357
88,goes without saying that,1,4,goes,without,saying,that,-6.09357
89,without saying that humans,1,4,without,saying,that,humans,-6.09357
90,saying that humans mammals,1,4,saying,that,humans,mammals,-6.09357
91,that humans mammals identifiable,1,4,that,humans,mammals,identifiable,-6.09357


#### 2 words back

In [17]:
k = n-1
ngram_3 = data[data['len']==k].iloc[:]
for i in range(k):
    ngram_3[f'w_{k-i-1}'] = ngram_3['words'].apply(lambda x: x.split()[i])
ngram_3[f'log_P(w_k-2|w_k)'] = np.log(ngram_3['nu'] / ngram_3['nu'].sum())
ngram_3.head()

,words,nu,len,w_2,w_1,w_0,log_P(w_k-2|w_k)
58,it goes without,1,3,it,goes,without,-6.137727
59,goes without saying,1,3,goes,without,saying,-6.137727
60,without saying that,1,3,without,saying,that,-6.137727
61,saying that humans,1,3,saying,that,humans,-6.137727
62,that humans mammals,1,3,that,humans,mammals,-6.137727


#### 1 words back

In [18]:
k = n-2
ngram_2 = data[data['len']==k].iloc[:]
for i in range(k):
    ngram_2[f'w_{k-i-1}'] = ngram_2['words'].apply(lambda x: x.split()[i])
ngram_2[f'log_P(w_k-1|w_k)'] = np.log(ngram_2['nu'] / ngram_2['nu'].sum())
ngram_2.head()

,words,nu,len,w_1,w_0,log_P(w_k-1|w_k)
28,it goes,1,2,it,goes,-6.180017
29,goes without,1,2,goes,without,-6.180017
30,without saying,1,2,without,saying,-6.180017
31,saying that,1,2,saying,that,-6.180017
32,that humans,1,2,that,humans,-6.180017


#### Word

In [19]:
k = n-3
ngram_1 = data[data['len']==k].iloc[:]
for i in range(k):
    ngram_1[f'w_{k-i-1}'] = ngram_1['words'].apply(lambda x: x.split()[i])
ngram_1[f'log_P(w_0)'] = np.log(ngram_1['nu'] / ngram_1['nu'].sum())
ngram_1.head()

,words,nu,len,w_0,log_P(w_0)
0,it,3,1,it,-5.121978
1,goes,1,1,goes,-6.220590
2,without,1,1,without,-6.220590
3,saying,1,1,saying,-6.220590
4,that,20,1,that,-3.224858


#### Concatenation

In [20]:
full = pd.concat((ngram_1,ngram_2,ngram_3,ngram_4)).fillna(0)

In [21]:
full.head()

,words,nu,len,w_0,log_P(w_0),w_1,log_P(w_k-1|w_k),w_2,log_P(w_k-2|w_k),w_3,log_P(w_k-3|w_k)
0,it,3,1,it,-5.121978,0,0.0,0,0.0,0,0.0
1,goes,1,1,goes,-6.220590,0,0.0,0,0.0,0,0.0
2,without,1,1,without,-6.220590,0,0.0,0,0.0,0,0.0
3,saying,1,1,saying,-6.220590,0,0.0,0,0.0,0,0.0
4,that,20,1,that,-3.224858,0,0.0,0,0.0,0,0.0


In [22]:
full.shape

(1481, 11)

### Testing Predicting Word

In [23]:
def predict_next_word(full, word):
    d = full[((full['w_1']==word) | (full['w_2']==word) | (full['w_3']==word))]
    e = full[(full['w_0'] == word) & (full['len']==1)]['log_P(w_0)'].values + d['log_P(w_k-1|w_k)']+d['log_P(w_k-2|w_k)']+d['log_P(w_k-3|w_k)']
    try:
        result = full.iloc[e.index[np.argmax(e)]]['w_0']
        return result
    except:
        return f"Word {word} is not in vocalbulary"

In [24]:
predict_next_word(full, 'have') 

'see'

Не предлог - радует. Проверим на остальных примерах.

In [25]:
words = ['that', 'top', 'spherical', 'and', 'in', 'goes', 'located', 'be', 'with', 'as', 'way', 'have']

next_word = [(word, predict_next_word(full, word = word)) for word in words]
no_preparation_result = pd.DataFrame(data = next_word, columns = ['word', 'next_word'])
no_preparation_result

,word,next_word
0,that,spherical
1,top,and
2,spherical,top
3,and,in
4,in,text
5,goes,located
6,located,of
7,be,associated
8,with,that
9,as,horizontal


Данный метод, оценивающий вероятнотсь появления слова на основании появления m-предыдущих слов, весьма хорошо справляется со своей работой.

In [20]:
word = 'with'

for i in range(6):
    print(word, end=' ')
    word = predict_next_word(full, word)
    
print(word)

with that spherical top and in text
